### Algunas funciones para el curso de ESMA 3016
###  Edgar Acuna

In [27]:
#Haciendo una funcion que haga la tabla completa de distribucion de frecuencias
def tablafreq(datos,str):
    """
    :param datos: Es el nombre de la base de datos
    :param str: Es el nombre de la variable a usar de la base de datos.
    """
    import pandas as pd
    tab1=pd.crosstab(index=datos[str],columns="counts",colnames=['variable'])
    tab1['frec.relat.porc']=tab1*100/tab1.sum()
    tab1['frec.acum']=tab1.counts.cumsum()
    tab1['frec.relat.porc.acum']=tab1['frec.acum']*100/tab1['counts'].sum()
    return tab1;

In [28]:
#Haciendo una funcion que haga la tabla completa de distribucion de frecuencias para datos agrupados
def tablafreqag(datos,str,k):
    """
    :param datos: Es el nombre de la base de datos
    :param str: Es el nombre de la variable a usar de la base de datos.
    :param k: Numero de intervalos
    """
    import pandas as pd
    m=min(datos[str])
    M=max(datos[str])
    ancho=(M-m)/k
    var1=np.array(datos[str])
    cortes1=np.linspace(m,M,num=k+1)
    a, b=pd.cut(var1,bins=cortes1,include_lowest=True, right=True, retbins=True)
    tablag=pd.crosstab(a,columns='counts')
    tablag['frec.relat.porc']=tablag*100/tablag.sum()
    tablag['frec.acum']=tablag.counts.cumsum()
    tablag['frec.relat.porc.acum']=tablag['frec.acum']*100/tablag['counts'].sum()
    tablag.round(3)
    return tablag;

In [29]:
def IC_media(data, sigma, confianza=0.95):
    # Esta es una funcion que hace el intervalo de confianza para la media poblacional
    """
    :param data: Es el nombre de la base de datos
    :param sigma: Es el valor asumido de la desviacion estandar poblacional
    :param confianza: El el nivel de confianza del intervalo. debe ser un numero cercano a1.
    """
    import numpy as np
    from scipy.stats  import norm,t
    from math import sqrt,factorial,pi   
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), sigma/sqrt(n)
    h = se * norm.ppf((1+confianza)/2)
    print "El intervalo de confianza es:", m-h, m+h
    return

In [30]:
def ztest(data,mu_0,sigma,alternativa="no igual a"):
    # Haciendo una funcion que halle p-value de la prueba estadistica de Z par la media poblalcional
    """
    :param data: Es el nombre de la base de datos
    :param mu_0: Valor asumiedo de la media poblacional en la hipotesis nula 
    :param sigma: Es el valor asumido de la desviacion estandar poblacional
    :param confianza: El el nivel de confianza del intervalo. debe ser un numero cercano a1.
    """
    import numpy as np
    from scipy.stats  import norm,t
    from math import sqrt,factorial,pi
    #Esta funcion calcula el p-value de la prueba z
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), sigma/sqrt(n)
    print "media",m
    test=(m-mu_0)/se
    if alternativa=="mayor que":
        p_value=1-norm.cdf(test)
    elif alternativa=="menor que":
        p_value=norm.cdf(test)
    else:
        p_value=2*(1-norm.cdf(abs(test)))
    print "El p-value es:", p_value    
    return

In [31]:
def boot_IC_perc(data,nboot,level):
    #IC por bootstrapping metodo de percentiles
    """
    :param data: Es el nombre de la base de datos
    :param nboot: Es el numero de muestras bootstrap 
    :param level: Es el nivel de confianza y esta dado en porcentaje
    """
    nd=len(data)
    b=[np.random.choice(data,nd).mean() for _ in range(nboot)] 
    liminf=np.percentile(b,(100-level)/2.0)
    limsup=np.percentile(b,(100+level)/2.0)
    return liminf,limsup

In [32]:
def stderror_boot(data,nboot,estimado="mean"):
    #Esta funcion calcula el error estándar para la media y la mediana
    # usando Bootsrapping con un numero de muestras igual a nboot. 
    """
    :param data: Es el nombre de la base de datos
    :param nboot: Es el numero de muestras bootstrap
    :param estimado: "mean" o "median"
    """
    nd=len(data) 
    if estimado=="mean":
        b=[np.random.choice(data,nd).mean() for _ in range(nboot)] 
    else:
        b=[np.median(np.random.choice(data,nd)) for _ in range(nboot)] 
    return np.std(b,ddof=1)

In [33]:
def boot_ICt(data,nboot,level):
    #calcula un intervalo de confianza para la media poblacional cuando la muestra x es pequeña 
    #nboot=numero de muestras bootstrap 
    """
    :param data: Es el nombre de la base de datos
    :param nboot: Es el numero de muestras bootstrap
    :param level: El el nivel de confianza del intervalo, debe estar dado en procentaje.
    """
    import math as m
    #level=Nivel de confianza en porcentaje 
    nd=len(data)
    xmean = np.mean(data) 
    se=np.std(data,ddof=1)/m.sqrt(nd) 
    bt=[(np.mean(np.random.choice(data,nd))-xmean)*m.sqrt(nd)/np.std(np.random.choice(data,nd)) for _ in range(nboot)] 
    error1 = np.percentile(bt, (100-level)/2.0) * se
    error2 = np.percentile(bt, (100+level)/2.0) * se
    limsup=xmean+error2
    liminf=xmean+error1 
    return liminf, limsup